<a href="https://colab.research.google.com/github/Jegovila/Computo-Evolutivo/blob/main/3%20Algoritmos%20Gen%C3%A9ticos/C%C3%B3digos/Algoritmos_Gen%C3%A9ticos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget -nc https://raw.githubusercontent.com/Jegovila/Computo-Evolutivo/main/3%20Algoritmos%20Gen%C3%A9ticos/C%C3%B3digos/Plot_Surf.py
!wget -nc https://raw.githubusercontent.com/Jegovila/Computo-Evolutivo/main/3%20Algoritmos%20Gen%C3%A9ticos/C%C3%B3digos/Plot_Contour.py

import matplotlib.pyplot as plt
import numpy as np
from Plot_Surf import *
from Plot_Contour import *
from IPython import display

# Funciones

## Selección

In [ ]:
def Seleccion(aptitud, algoritmo):
    if algoritmo == 'ga' or algoritmo == 'elitism':
      aptitud_total = np.sum(aptitud)
      N = (aptitud.size)

      r = np.random.rand()
      p_sum = 0

      for i in range(N):
          p_sum = p_sum + aptitud[i] / aptitud_total

          if p_sum >= r:
              n = i
              return n

      n = N
      return n

    elif algoritmo == 'rank':

      Idx = np.argsort(aptitud)
      Idx = Idx[::-1]
      N = (aptitud.size)

      rank = np.arange(N,-1,-1)
      rank_total = np.sum(rank)

      r = np.random.rand()
      p_sum = 0

      for i in range(N):
        p_sum = p_sum + rank[i] / rank_total

        if p_sum >= r:
            n = Idx[i]
            return n

      n = N
      return n

## Cruza

In [ ]:
def Cruza(x1, x2, algoritmo):

  if algoritmo == 'elitism' or algoritmo == 'rank':
    D = x1.size
    pc = np.random.randint(0, D)

    y1 = x1.copy()
    y2 = x2.copy()

    y1[pc:] = x2[pc:]
    y2[pc:] = x1[pc:]
    return y1, y2

  elif algoritmo == 'ga':
    D = x1.size
    pc = np.random.randint(0, D)

    y1 = x1.copy()
    y2 = x2.copy()

    y1[pc:] = x2[pc:]
    y2[pc:] = x1[pc:]
    return y1, y2


## Mutación

In [ ]:
def Mutacion(y, pm, x=None, xu=None, algoritmo='rank'):
  if algoritmo == 'ga':
    D, N = y.shape

    for i in range(N):
        for j in range(D):
            if np.random.rand() < pm:
                y[j,i] = xl[j] + (xu[j] - xl[j]) * np.random.rand()
    return y

  elif algoritmo == 'elitism' or algoritmo == 'rank':
    D, N = y.shape

    for i in range(N):
        for j in range(D):
            if np.random.rand() < pm:
                y[j,i] = y[j,i] + np.random.normal(0, 1)
    return y


# Parámetros

In [ ]:
f = lambda x, y: (x-2)**2 + (y-2)**2

xl = np.array([-5, -5])
xu = np.array([5, 5])

N = 50
E = 10
D = 2
G = 100
pm = 0.01

# Algoritmos Genéticos

Elegir el algoritmo genético

In [ ]:
algoritmo = 'ga'
#algoritmo = 'elitism'
#algoritmo = 'rank'

In [ ]:
x = np.zeros((D,N))
aptitud = np.zeros(N)

for i in range(N):
    x[:, i] = xl + (xu - xl) * np.random.rand(D)

for g in range(G):
    display.display(plt.gcf())
    display.clear_output(wait=True)
    plot_contour(f, x, xl, xu)

    for i in range(N):
        fx = f(x[0, i], x[1, i])

        if fx >= 0:
            aptitud[i] = 1 / (1 + fx)
        else:
            aptitud[i] = 1 + np.abs(fx)

    if algoritmo == 'ga' or algoritmo == 'rank':
      y = np.zeros((D, N))

      for i in range(0, N, 2):
          r1 = Seleccion(aptitud, algoritmo)
          r2 = r1

          while r2 == r1:
              r2 = Seleccion(aptitud, algoritmo)

          y1, y2 = Cruza(x[:, r1], x[:, r2], algoritmo)

          y[:, i] = y1
          y[:, i+1] = y2

      y = Mutacion(y, pm, xl, xu, algoritmo)
      x = y.copy()

    elif algoritmo == 'elitism':
      y = np.zeros((D, N-E))

      for i in range(0, N-E, 2):
          r1 = Seleccion(aptitud, algoritmo)
          r2 = r1

          while r2 == r1:
              r2 = Seleccion(aptitud, algoritmo)

          y1, y2 = Cruza(x[:, r1], x[:, r2], algoritmo)

          y[:, i] = y1
          y[:, i+1] = y2

      y = Mutacion(y, pm, algoritmo)
      Idx = np.argsort(aptitud)
      Idx = Idx[::-1]
      x = np.concatenate((y,x[:, Idx[0:E]]),axis=1)

for i in range(N):
    fx = f(x[0, i], x[1, i])

    if fx >= 0:
        aptitud[i] = 1 / (1 + fx)
    else:
        aptitud[i] = 1 + abs(fx)

i_mejor = np.argmax(aptitud)
plot_surf(f, x, xl, xu, i_mejor)

print("Mínimo global en x=", x[0,i_mejor], " y=", x[1,i_mejor], " f(x,y)=", f(x[0,i_mejor], x[1,i_mejor]))